In [86]:
import numpy as np
import torch.nn.functional as F
import torch

In [87]:
oc = 8
ic = 16
c = 16
n = 4
k = 3

np.random.seed(343)
w = np.random.randint(0, 16, (oc, ic, k, k)) - 8 # oc, ic, kh, kw
x = np.random.randint(0, 4, (ic, n, n)) # ic, nh, nw
x = np.pad(x, ((0,0), (1,1), (1,1)))

w = torch.tensor(w)
x = torch.tensor(x)

psum = F.conv2d(x, w)

# to test that it fails when expected
# w[0,0,0,0] -= 1

with open('x2b.txt', 'w+') as f:
    for val in x.numpy().transpose(1,2,0).flatten():
        f.write(f"{val:02b}\n")

with open('w2b.txt', 'w+') as f:
    for ky in range(3):
        for kx in range(k):
            for oc in range(8):
                for i in range(2):
                    for ic in range(i,16,2):
                        val = w[oc,ic,ky,kx]
#    for val in w.numpy().transpose(2,3,0,1).flatten():
                        f.write(f"{val&0xF:04b}\n")

with open('psum2b.txt', 'w+') as f:
    for val in psum.reshape(8,n*n).numpy().T.flatten():
        f.write(f"{val&0xFFFF:016b}\n")

In [88]:
with open('x2b_os.txt', 'w+') as f:
    for ic in range(0,16,2):
        for ky in range(k):
            for kx in range(k):
                _ = 0
                vals = []
                for ny in range(n):
                    for nx in range(n):
                        vals.append(x[ic,ny+ky,nx+kx])
                        vals.append(x[ic+1,ny+ky,nx+kx])
                        _ += 1
                        if _ == 8: break
                    if _ == 8: break
                for val in vals[::-1]:
                    f.write(f"{val:02b}")
                f.write('\n')

with open('w2b_os.txt', 'w+') as f:
    for ic in range(0,16,2):
        for ky in range(k):
            for kx in range(k):
                for _ in range(2):
                    for oc in range(min(8, c)):
                        f.write(f"{w[7-oc,ic+_,ky,kx]&0xF:04b}")
                    f.write('\n')

In [89]:
psum_debug = torch.tensor(np.zeros((k*k, c, n, n))) # kij, oc, ny, nx
for ky in range(k):
    for kx in range(k):
        break
        psum_debug[ky*k+kx] = (F.conv2d(x[:,ky:ky+n,kx:kx+n], w[:,:,ky:ky+1,kx:kx+1]))


In [90]:
def hp(arr, digits=4, rev=True):
    try: arr = arr.numpy()
    except: pass
    assert len(arr.shape) <= 2
    if len(arr.shape) == 1: arr = [arr]
    for line in arr:
        for item in line[::-1 if rev else 1]:
            s = hex(int(item) & ((1<<(4*digits))-1))[2:].zfill(digits)
            print(s, end=' ')
        print()